<a href="https://colab.research.google.com/github/Johoodcoder/CS490Project/blob/hood/Notebooks/CS490ProjectHugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Non-preinstalled module installs

In [ ]:
!pip install pytorch-pretrained-bert

In [ ]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from collections import Counter

In [ ]:
# Load dataset
# df = pd.read_csv("condensed_fake_real_news_SANITIZED.csv")
df = pd.read_csv("LIARPLUSTrainSanitized.csv")
df = df[['text', 'type']]
print(len(df))

10234


In [ ]:
df = df[df['type'].isin(['fake', 'real'])]
# Scramble data indexes from dataset. Random_state is a seed.
df = df.dropna()
df = df.sample(frac=1, random_state = 23).reset_index(drop=True)

print(Counter(df['type'].values))

Counter({'real': 7347, 'fake': 2803})


In [ ]:
train_data_df = df.head(8000)
test_data_df = df.tail(2000)
print(train_data_df)

                                                   text  type
0     Bike Austin said: "Bike lanes and sidewalks ha...  real
1     But his full-bodied explanation at the debate ...  fake
2     Alee Lockman, Bruuns spokeswoman, says the con...  real
3     Clinton said, "the U. S. military footprint in...  real
4     The Florida Democratic Party said, "Marco Rubi...  fake
...                                                 ...   ...
7995  Wheeler and Soglin heaped praise on law enforc...  fake
7996  Boehner said, "This is the same bill we voted ...  real
7997  Though if Mr. Hiaasen comes up with that magic...  fake
7998  Estimated ammunition inventories for both agen...  real
7999  The $250 in question is a calculation based on...  real

[8000 rows x 2 columns]


In [ ]:
train_data = []
for index, row in train_data_df.iterrows():
    train_data.append({'text': row['text'], 'type': row['type']})

test_data = []
for index, row in test_data_df.iterrows():
    test_data.append({'text': row['text'], 'type': row['type']})

In [ ]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['type']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['type']), test_data)))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [ ]:
# If value == fake then make it true. Otherwise false.
train_y = np.array(train_labels) == 'fake'
test_y = np.array(test_labels) == 'fake'

train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((8000,), (2000,), 0.276625, 0.2765)

In [ ]:
# Input masks differentiate padding tokens from legitimate data token. 1 == data, 0 == padding
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

In [ ]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

# Testing datatypes for speed and compatability
# ----------------------------------------- CUDA -----------------------------------------------------
# train_masks_tensor = train_masks_tensor.to('cuda')
# test_masks_tensor = test_masks_tensor.to('cuda')

# train_tokens_tensor = train_tokens_tensor.to('cuda')
# test_tokens_tensor = test_tokens_tensor.to('cuda')

train_y_tensor = train_y_tensor.to('cuda')
test_y_tensor = test_y_tensor.to('cuda')

# ----------------------------------------- CUDA LONG -----------------------------------------------------
cuda = torch.device('cuda')
train_masks_tensor = train_masks_tensor.to(cuda, dtype = torch.long)
test_masks_tensor = test_masks_tensor.to(cuda, dtype = torch.long)

train_tokens_tensor = train_tokens_tensor.to(cuda, dtype = torch.long)
test_tokens_tensor = test_tokens_tensor.to(cuda, dtype = torch.long)

# train_y_tensor = train_y_tensor.to(cuda, dtype = torch.long)
# test_y_tensor = test_y_tensor.to(cuda, dtype = torch.long)

In [ ]:
BATCH_SIZE = 8
EPOCHS = 1

train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

num_labels = 1

In [ ]:
bert_clf = BertForSequenceClassification(config, num_labels)
bert_clf.to('cuda')
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)

for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):

        token_ids, masks, labels = tuple(t for t in batch_data)
        
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        # The new model has slightly different outputs. A sigmoid() is applied to probas to bound between 0 and 1
        batch_loss = loss_func(logits.sigmoid(), labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
target_names = ['Real News', 'Fake News']
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        # The new model has slightly different outputs. A sigmoid() is applied to logits to bound between 0 and 1
        loss = loss_func(logits.sigmoid(), labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted, target_names=target_names))

In [ ]:
for testVal, predVal, in zip(test_y, bert_predicted):
  print(f'{testVal}\t{predVal}')